In [1]:
import pandas as pd
import numpy as np
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
#pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import datetime 

In [2]:
tx_df = pd.read_table("./data/TXN_DATA.TXT",sep = ",",
    names =["ID","MccCode","SIGNFLAG","TxAmt","sTxAmt","TcCode","No","TxDate"])

In [3]:
#trainset = pd.read_table("./data/m0_200411.txt",sep = ",")
testset =pd.read_table("./data/m0_200412.txt",sep = ",")
trainset = testset

In [4]:
trainset.shape

(63532, 32)

In [8]:
#查看男女性占比
from pyecharts.charts import Pie

attr = ["男性","女性"]
v1 = [trainset["SEX"].value_counts()[1],trainset["SEX"].value_counts()[2]]
#pie = Pie("性别分布")
pie = Pie()
pie.add("", attr, v1,)

pie


TypeError: add() takes 3 positional arguments but 4 were given

In [6]:
#查看教育占比
from pyecharts import Pie

attr = ["硕博及以上","本科","高中","初中及以下"]
v1 = [trainset["EDUCATION"].value_counts()[-1],
      trainset["EDUCATION"].value_counts()[1],
      trainset["EDUCATION"].value_counts()[0],
      sum(trainset["EDUCATION"].value_counts()[2:7])]
pie = Pie("教育分布")
pie.add("", attr, v1, is_label_show=True)

pie


In [7]:
#查看婚姻状态占比
from pyecharts import Pie

attr = ["已婚","未婚","离婚"]
v1 = [trainset["MARRIAGE"].value_counts()[1],
      trainset["MARRIAGE"].value_counts()[2],
      trainset["MARRIAGE"].value_counts()[3]]
pie = Pie("婚姻分布")
pie.add("", attr, v1, is_label_show=True)

pie

In [9]:
#查看年龄分布
import matplotlib.pyplot as plt
plt.hist(trainset["AGE"], rwidth=0.9)
plt.show()

<Figure size 640x480 with 1 Axes>

In [5]:
#定义支付码填充函数
def ex_str_pay_code(y):
    e = list()
    c = list()
    n_3 = 0
    n_24 = 0
    for x in list(y):
        if x in ["z","Z","b","B"]:
            e.append("0")
        elif x.isspace():
            e.append("0")
        else:
            e.append(x)
            n_24 = n_24 + 1
    for  i in range(1,9) :
        n_3 = 0
        for j in list(y)[4*(i-1):4*i-1]:
            if j in ['2','3','4','5','6','7','8','9']:
                n_3 = n_3 + 1
        c.append(str(n_3))
     #24位支付码   
    pay_code = ''.join(e)
    #每三个月逾期次数
    overdue_cnt_3 = ''.join(c)
    #最大逾期月份
    max_overdue_mon = max(e)
    return  [pay_code,overdue_cnt_3,max_overdue_mon]

#调整函数
def ex_repay_rate(x):
    if x<=2 and x>=-2:
        return x
    elif x>2:
        return 2
    else:
        return -2
#实动月份
def use_mon(x):
    a = list()
    for i in list(x):
        if i not in ("Z","z"," "):
            a.append(i)
    return len(a)

In [ ]:
#定义星座
def xingzuo(month, day):
    day = int(month)*100 + int(day)
    if day >= 123 and day <=218:
        return('1:水瓶座')
    elif day <= 320:
        return('2:双鱼座')
    elif day <= 419:
        return('3:白羊座')
    elif day <= 520:
        return('4:金牛座')
    elif day <= 621:
        return('5:双子座')
    elif day <= 722:
        return('6:巨蟹座')
    elif day <= 822:
        return('7:狮子座')
    elif day <= 922:
        return('8:处女座')
    elif day <= 1023:
        return('9:天秤座')
    elif day <= 1122:
        return('10:天蝎座')
    elif day <= 1221:
        return('11:射手座')
    else:
        return('12:摩羯座')

In [6]:
#计算前6个月各月的额度使用率
trainset["LIMIT_RATE_1"]=[*map(lambda x,y:x/y,trainset["BILL_AMT1"],trainset["LIMIT_BAL"])]
trainset["LIMIT_RATE_2"]=[*map(lambda x,y:x/y,trainset["BILL_AMT2"],trainset["LIMIT_BAL"])]
trainset["LIMIT_RATE_3"]=[*map(lambda x,y:x/y,trainset["BILL_AMT3"],trainset["LIMIT_BAL"])]
trainset["LIMIT_RATE_4"]=[*map(lambda x,y:x/y,trainset["BILL_AMT4"],trainset["LIMIT_BAL"])]
trainset["LIMIT_RATE_5"]=[*map(lambda x,y:x/y,trainset["BILL_AMT5"],trainset["LIMIT_BAL"])]
trainset["LIMIT_RATE_6"]=[*map(lambda x,y:x/y,trainset["BILL_AMT6"],trainset["LIMIT_BAL"])]

In [7]:
#计算前6个月各月的还贷比
trainset["REPAY_RATE_1"]=[*map(lambda x,y:ex_repay_rate(x/y) if y!=0 else 0,trainset["PAY_AMT1"],trainset["BILL_AMT1"])]
trainset["REPAY_RATE_2"]=[*map(lambda x,y:ex_repay_rate(x/y) if y!=0 else 0,trainset["PAY_AMT2"],trainset["BILL_AMT2"])]
trainset["REPAY_RATE_3"]=[*map(lambda x,y:ex_repay_rate(x/y) if y!=0 else 0,trainset["PAY_AMT3"],trainset["BILL_AMT3"])]
trainset["REPAY_RATE_4"]=[*map(lambda x,y:ex_repay_rate(x/y) if y!=0 else 0,trainset["PAY_AMT4"],trainset["BILL_AMT4"])]
trainset["REPAY_RATE_5"]=[*map(lambda x,y:ex_repay_rate(x/y) if y!=0 else 0,trainset["PAY_AMT5"],trainset["BILL_AMT5"])]
trainset["REPAY_RATE_6"]=[*map(lambda x,y:ex_repay_rate(x/y) if y!=0 else 0,trainset["PAY_AMT6"],trainset["BILL_AMT6"])]

In [9]:
#求得支付码对应的位数，求得用户使用月份
trainset['ALL_MON'] = [*map(lambda x:int(len(x.split(' ')[0])),trainset["PAY_COND"])]
#将这个数据填充为24位，Z\0\B均替换为0，不满24位的右边补0
#每三个月的逾期次数
trainset['OVERDUE_CNT_3'] = [*map(lambda x: int(ex_str_pay_code(x)[1]),trainset["PAY_COND"])]
#最大逾期月份
trainset['OVERDUE_MAX_MON'] = [*map(lambda x: int(ex_str_pay_code(x)[2]),trainset["PAY_COND"])]
#获取实动月份
trainset["USE_MON"] = [*map(lambda x:use_mon(x),trainset["PAY_COND"])]

In [10]:
trainset.columns

Index(['ID_NO_IDENT', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'JOB_TYPE', 'MARRIAGE',
       'ZIP_CODE', 'BIRTHDAY', 'AGE', 'CASE_NO', 'CASE_FROM', 'PAY_COND',
       'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1',
       'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
       'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'SCORING', 'CUST_TYPE', 'LIMIT_RATE_1', 'LIMIT_RATE_2', 'LIMIT_RATE_3',
       'LIMIT_RATE_4', 'LIMIT_RATE_5', 'LIMIT_RATE_6', 'REPAY_RATE_1',
       'REPAY_RATE_2', 'REPAY_RATE_3', 'REPAY_RATE_4', 'REPAY_RATE_5',
       'REPAY_RATE_6', 'ALL_MON', 'OVERDUE_CNT_3', 'OVERDUE_MAX_MON',
       'USE_MON'],
      dtype='object')

In [11]:
#画箱体图
from matplotlib import pyplot as plt
df=pd.DataFrame({"Good":trainset["ALL_MON"][trainset["CUST_TYPE"]=="G"],
                 "Bad":trainset["ALL_MON"][trainset["CUST_TYPE"]=="B"]})
df.boxplot()
plt.show()

<Figure size 640x480 with 1 Axes>

In [13]:
trainset.describe()

,ID_NO_IDENT,LIMIT_BAL,SEX,MARRIAGE,ZIP_CODE,BIRTHDAY,AGE,CASE_NO,CASE_FROM,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SCORING,LIMIT_RATE_1,LIMIT_RATE_2,LIMIT_RATE_3,LIMIT_RATE_4,LIMIT_RATE_5,LIMIT_RATE_6,REPAY_RATE_1,REPAY_RATE_2,REPAY_RATE_3,REPAY_RATE_4,REPAY_RATE_5,REPAY_RATE_6,ALL_MON,OVERDUE_CNT_3,OVERDUE_MAX_MON,USE_MON
count,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,56244.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,0.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00,63532.00
mean,1102508.73,168976.23,1.59,1.57,417.50,19697042.64,35.36,8555220513732.71,6.12,49579.40,47401.54,45270.61,42543.72,38145.76,36843.86,5782.91,5691.60,5625.93,4785.55,4583.67,4710.74,nan,0.41,0.39,0.37,0.34,0.30,0.29,0.29,0.28,0.27,0.23,0.24,0.25,18.95,3879578.20,1.30,14.71
std,644080.94,130822.89,0.49,0.52,265.68,92951.25,9.30,1898587086426.05,4.80,73549.02,71629.17,69351.02,65872.24,60774.63,59232.40,17990.24,18118.39,18219.23,17510.83,15152.59,16093.03,nan,0.41,0.41,0.40,0.38,0.35,0.34,0.60,0.61,0.60,0.56,0.56,0.58,6.66,8465012.46,0.95,8.34
min,7.00,10000.00,1.00,0.00,0.00,19260318.00,21.00,4072100004.00,1.00,-154973.00,-166648.00,-179019.00,-182415.00,-186386.00,-209051.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,-1.02,-1.45,-1.40,-1.37,-1.18,-1.03,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,0.00,0.00,0.00,0.00
25%,542313.25,50000.00,1.00,1.00,231.00,19640306.00,28.00,9008132808287.50,2.00,2550.00,1792.00,1275.50,853.75,626.75,396.00,482.75,380.00,41.00,0.00,0.00,0.00,nan,0.02,0.01,0.01,0.01,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,14.00,0.00,1.00,8.00
50%,1091100.00,150000.00,2.00,2.00,330.00,19711003.00,34.00,9110302409158.00,5.00,19607.50,18626.00,18087.50,17131.50,15011.00,13693.50,2000.00,2000.00,1880.00,1242.00,1200.00,1290.00,nan,0.27,0.24,0.20,0.17,0.14,0.12,0.05,0.05,0.04,0.04,0.04,0.04,23.00,0.00,1.00,16.00
75%,1662860.00,240000.00,2.00,2.00,702.00,19770726.00,41.00,9207243722171.50,11.00,63541.25,60169.00,57153.75,52601.25,47832.50,46514.25,5000.00,5000.00,5000.00,4000.00,4000.00,4000.00,nan,0.82,0.79,0.75,0.67,0.57,0.55,0.22,0.20,0.17,0.11,0.11,0.13,24.00,1000000.00,2.00,24.00
max,2205879.00,2000000.00,2.00,3.00,983.00,19841109.00,79.00,9306155922001.00,21.00,947995.00,1001477.00,991631.00,706864.00,904519.00,927171.00,1002035.00,1227082.00,889043.00,904549.00,559272.00,942817.00,nan,4.38,4.49,10.69,4.74,4.94,4.26,2.00,2.00,2.00,2.00,2.00,2.00,24.00,33333300.00,9.00,24.00


In [14]:
tx_df.describe()

KeyboardInterrupt: 

In [14]:
#处理交易数据
tx_df["TxAmt"]=tx_df["TxAmt"]/100
tx_df["sTxAmt"]=tx_df["sTxAmt"]/100
tx_df.head(10)

,ID,MccCode,SIGNFLAG,TxAmt,sTxAmt,TcCode,No,TxDate
0,2130081.00,5311,+,6.00,6.00,40,4579530401421903,20040510
1,843083.00,5411,+,3.00,3.00,40,4579522401255605,20040522
2,1432605.00,5968,+,51.81,51.81,40,4514453605964206,20040919
3,21431.00,5311,+,30.60,30.60,40,4579532902001507,20040507
4,166293.00,4722,+,97.92,97.92,40,4579531600090903,20040624
5,1789252.00,5541,+,0.70,0.70,40,4579522002578900,20040815
6,2035955.00,0,-,3.99,-3.99,43,4514453610086805,20040729
7,949283.00,7011,+,33.60,33.60,40,5179510000220600,20040704
8,2201572.00,6010,+,150.00,150.00,30,4579532409524001,20041122
9,1105981.00,5311,+,8.99,8.99,40,4579522502861509,20041111


In [15]:
tx_df.groupby("SIGNFLAG").count()

,ID,MccCode,TxAmt,sTxAmt,TcCode,No,TxDate
SIGNFLAG,,,,,,,
+,16984892,16985500,16985500,16985500,16985500,16985500,16985500
-,1882547,1882579,1882579,1882579,1882579,1882579,1882579


In [17]:
#获取各个ID的消费次数和消费金额
txset=tx_df[tx_df["SIGNFLAG"]=="+"]
ex_cnt = txset.groupby("ID")["ID"].size()
ex_cnt.name = 'Count'
#金额
ex_amt = txset.groupby("ID")["TxAmt"].sum()
ex_amt.name = "Amount"

In [18]:
#获取各个ID的入账次数和入账金额
txset1=tx_df[tx_df["SIGNFLAG"]=="-"]
ex_cnt1 = txset1.groupby("ID")["ID"].size()
ex_cnt1.name = "sCcount"
#金额
ex_amt1 = txset1.groupby("ID")["TxAmt"].sum()
ex_amt1.name = "sAmount"

In [22]:
#获取各个ID的交易月份数
tx_df["Tx_Mon"]=[*map(lambda x:str(x)[0:6],tx_df["TxDate"])]
tx_count = tx_df.groupby(["ID","Tx_Mon"])["ID"].count()
tx_mon_cnt = tx_count.to_frame(name = "A").reset_index().groupby("ID").count()["Tx_Mon"]
tx_mon_cnt.name = "Tx_Mon"

In [ ]:
#月均出账次数
#月均出账金额
#月均入账次数
#出账笔均金额
#交易月份数
#资金需求
#近3月出账次数
#近3月出账金额
#近3月入账次数
#消费渠道数
#高风险MCC码
#

In [ ]:
#读入MccCode字段
MccCode = pd.read_csv("./Data/MccCode.csv",sep = ",")

#加入Mcc码的交易中类
tx_df1 = pd.merge(tx_df,MccCode,how = "left",left_on = "MccCode",right_on = "MCC")

In [16]:
#对未匹配上的数据填充“空”
tx_df1["中类"] = tx_df1['中类'].fillna("空")
#将好坏客户与交易类型连接起来
trainset1 = trainset[["ID_NO_IDENT","CUST_TYPE"]]
tx_df_2 = tx_df1[["ID","中类"]]
traindata = pd.merge(trainset1 ,tx_df_2,how = "right",left_on = "ID_NO_IDENT",right_on ="ID")

In [18]:
#求得好坏客户不同类别的交易数量
left = traindata.groupby(["CUST_TYPE","中类"])["ID"].count().to_frame(name = "lei_count").reset_index()
#求得不同类别的交易数量（不分好坏客户）
right = left.groupby("中类")["lei_count"].sum().to_frame(name = "count").reset_index()
#合并一起求占比
traindata = pd.merge(left,right,how = "outer",left_on = "中类",right_on = "中类")
#Mcc占比为好坏客户不同类别的交易占比
traindata["Mcc_RATE"] = traindata["lei_count"]/traindata["count"]
#单独拿出坏客户查看
traindata_bad = traindata[traindata["CUST_TYPE"] =="B"]
#风险切分点为14%，故14%*0.85=12%以下为低风险，14%*1.15=16%以上为高风险，中间为中等风险
traindata[traindata["CUST_TYPE"] =="B"]["lei_count"].sum()/(traindata[traindata["CUST_TYPE"] =="B"]["lei_count"].sum()+traindata[traindata["CUST_TYPE"] =="G"]["lei_count"].sum())

0.14006453034631008

In [19]:
#计算坏客户中Mcc码的占比，切分交易风险类别
def Mcc_rate_bin(x):
    if x>=0.16:
        return "高"
    elif x>=0.12:
        return "中"
    else:
        return "低"

In [21]:
#获得高风险类别交易
traindata_bad["Mcc_bin"] = [*map(lambda x : Mcc_rate_bin(x),traindata_bad["Mcc_RATE"])]

NameError: name 'traindata_bad' is not defined

In [20]:
#获得风险交易字典值
risk_dict = traindata_bad[["中类","Mcc_bin"]]

NameError: name 'traindata_bad' is not defined

In [ ]:
#-------算出每一个客户中高风险交易占比--------

#将交易数据与风险字典进行匹配
tx_df2 = pd.merge(tx_df1,risk_dict,how = "left",left_on = "中类",right_on = "中类")
#空值用无风险填充
tx_df2["Mcc_bin"]=tx_df2["Mcc_bin"].fillna("无")
#获得每一个客户不同风险的交易笔数
risk_rate = tx_df2.groupby("ID")["Mcc_bin"].value_counts().to_frame(name = "count").reset_index()

In [ ]:
#获取高|中|低风险交易笔数
risk_H = pd.Series(risk_rate[risk_rate["Mcc_bin"]=="高"]["count"].values,
                   index = risk_rate[risk_rate["Mcc_bin"]=="高"]["ID"],name = "Hrisk_rate")
risk_M = pd.Series(risk_rate[risk_rate["Mcc_bin"]=="中"]["count"].values,
                   index = risk_rate[risk_rate["Mcc_bin"]=="中"]["ID"],name = "Mrisk_rate")
risk_L = pd.Series(risk_rate[risk_rate["Mcc_bin"]=="低"]["count"].values,
                   index = risk_rate[risk_rate["Mcc_bin"]=="低"]["ID"],name = "Lrisk_rate")
#获得所有交易笔数
tx_cnt = tx_df2.groupby("ID")["Mcc_bin"].count()
tx_cnt.name = "total_tx_cnt"

In [66]:
#获得各个风险交易的占比
risk_tx_info = pd.concat([ex_cnt,ex_amt,ex_cnt1,ex_amt1,tx_mon_cnt,risk_H,risk_M,risk_L,tx_cnt],axis=1)
#空值用0填充
risk_tx_info = risk_tx_info.fillna(0.0001)

In [88]:
ex_info = pd.DataFrame()

In [89]:
#求得每个客户中不同风险的占比
ex_info["Hrisk_rate"] = risk_tx_info["Hrisk_rate"]/risk_tx_info["total_tx_cnt"]
ex_info["Mrisk_rate"] = risk_tx_info["Mrisk_rate"]/risk_tx_info["total_tx_cnt"]
ex_info["Lrisk_rate"] = risk_tx_info["Lrisk_rate"]/risk_tx_info["total_tx_cnt"]


In [90]:
risk_tx_info.head()

,Count,Amount,sCcount,sAmount,Tx_Mon,Hrisk_rate,Mrisk_rate,Lrisk_rate,total_tx_cnt
ID,,,,,,,,,
1.00,7.00,2912.00,0.00,0.00,7,1.00,0.00,0.00,7
5.00,11.00,5280.00,0.00,0.00,5,0.09,0.55,0.36,11
6.00,6.00,10511.00,1.00,3.00,3,0.14,0.29,0.57,7
7.00,12.00,25713.00,1.00,5.00,3,0.15,0.62,0.23,13
8.00,4.00,1119.00,1.00,2.00,4,0.00,0.20,0.80,5


In [91]:
#月均消费次数
ex_info["XF_CNT_M"] = risk_tx_info["Count"]/risk_tx_info["Tx_Mon"]
#月均消费金额
ex_info["XF_AMT_M"] = risk_tx_info["Amount"]/risk_tx_info["Tx_Mon"]
#月均入账次数
ex_info["RZ_CNT_M"] = risk_tx_info["sCcount"]/risk_tx_info["Tx_Mon"]
#月均入账金额
ex_info["RZ_AMT_M"] = risk_tx_info["sAmount"]/risk_tx_info["Tx_Mon"]
#笔均消费金额
ex_info["XF_AMT_CNT"] = risk_tx_info["Amount"]/risk_tx_info["Count"]

In [92]:
ex_info.head()

,Hrisk_rate,Mrisk_rate,Lrisk_rate,XF_CNT_M,XF_AMT_M,RZ_CNT_M,RZ_AMT_M,XF_AMT_CNT
ID,,,,,,,,
1.00,0.14,0.00,0.00,1.00,416.00,0.00,0.00,416.00
5.00,0.01,0.05,0.03,2.20,1056.00,0.00,0.00,480.00
6.00,0.02,0.04,0.08,2.00,3503.67,0.33,1.00,1751.83
7.00,0.01,0.05,0.02,4.00,8571.00,0.33,1.67,2142.75
8.00,0.00,0.04,0.16,1.00,279.75,0.25,0.50,279.75


In [103]:
trainset.columns

Index(['ID_NO_IDENT', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'JOB_TYPE', 'MARRIAGE',
       'ZIP_CODE', 'BIRTHDAY', 'AGE', 'CASE_NO', 'CASE_FROM', 'PAY_COND',
       'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1',
       'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
       'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'SCORING', 'CUST_TYPE', 'LIMIT_RATE_1', 'LIMIT_RATE_2', 'LIMIT_RATE_3',
       'LIMIT_RATE_4', 'LIMIT_RATE_5', 'LIMIT_RATE_6', 'REPAY_RATE_1',
       'REPAY_RATE_2', 'REPAY_RATE_3', 'REPAY_RATE_4', 'REPAY_RATE_5',
       'REPAY_RATE_6', 'ALL_MON', 'OVERDUE_CNT_3', 'OVERDUE_MAX_MON',
       'USE_MON'],
      dtype='object')

In [104]:
trainset1 = trainset[['ID_NO_IDENT', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'JOB_TYPE', 'MARRIAGE',
       'ZIP_CODE',  'AGE',  'CASE_FROM',
       'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1',
       'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
       'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'CUST_TYPE', 'LIMIT_RATE_1', 'LIMIT_RATE_2', 'LIMIT_RATE_3',
       'LIMIT_RATE_4', 'LIMIT_RATE_5', 'LIMIT_RATE_6', 'REPAY_RATE_1',
       'REPAY_RATE_2', 'REPAY_RATE_3', 'REPAY_RATE_4', 'REPAY_RATE_5',
       'REPAY_RATE_6', 'ALL_MON', 'OVERDUE_CNT_3', 'OVERDUE_MAX_MON',
       'USE_MON']]

In [105]:
train = pd.merge(trainset1,ex_info,how = "left",left_on = "ID_NO_IDENT",right_index =True)

In [112]:
train = train.fillna(0)

In [113]:
train.isnull().sum()

ID_NO_IDENT        0
LIMIT_BAL          0
SEX                0
EDUCATION          0
JOB_TYPE           0
MARRIAGE           0
ZIP_CODE           0
AGE                0
CASE_FROM          0
PAY_1              0
PAY_2              0
PAY_3              0
PAY_4              0
PAY_5              0
PAY_6              0
BILL_AMT1          0
BILL_AMT2          0
BILL_AMT3          0
BILL_AMT4          0
BILL_AMT5          0
BILL_AMT6          0
PAY_AMT1           0
PAY_AMT2           0
PAY_AMT3           0
PAY_AMT4           0
PAY_AMT5           0
PAY_AMT6           0
SCORING            0
CUST_TYPE          0
LIMIT_RATE_1       0
LIMIT_RATE_2       0
LIMIT_RATE_3       0
LIMIT_RATE_4       0
LIMIT_RATE_5       0
LIMIT_RATE_6       0
REPAY_RATE_1       0
REPAY_RATE_2       0
REPAY_RATE_3       0
REPAY_RATE_4       0
REPAY_RATE_5       0
REPAY_RATE_6       0
ALL_MON            0
OVERDUE_CNT_3      0
OVERDUE_MAX_MON    0
USE_MON            0
Hrisk_rate         0
Mrisk_rate         0
Lrisk_rate   

In [106]:
train.shape

(62309, 53)

In [114]:
train.to_csv("traindata.csv",index = True)